In [1]:
import pandas as pd
import datetime

In [2]:
fakerole_df = pd.read_csv("01-tw-fakeroles.csv")

/Users/megankatsumi/tw_faker_data/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3553: DtypeWarning: Columns (2,8) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


# RDF transformation

In [3]:
#define key namespaces to be used in the mapping
famo = "http://ontology.eil.utoronto.ca/FAMO/famo/"
ex = "http://example.com/data/"
rdf = "http://www.w3.org/1999/02/22-rdf-syntax-ns#"
owl = "http://www.w3.org/2002/07/owl#"

#current datetime to be used for identifier suffix
dt_now = datetime.datetime.now().strftime("%d%b%Y_%H%M%S")

## Role mapping

**TODO** test for null values in all columns before mapping

**Note** for the purposes of this mapping, the role records are used to generate corresponding DH entities. Each Role record is represented as pinned to the corresponding DH entity (to instantiate the data set), and initialized with the wms entity number as canonical.

In [4]:
ttl_list = []
#prefixes
ttl_list.append(f"PREFIX famo: <{famo}>"+'\n')
ttl_list.append(f"PREFIX ex: <{ex}>"+'\n')
ttl_list.append(f"PREFIX rdf: <{rdf}>"+'\n')
ttl_list.append(f"PREFIX owl: <{owl}>"+'\n')

for i in range(0,len(fakerole_df)):
    #record instantiation and interpretation
    ttl_list.append(f"ex:wms_rolerec_{fakerole_df['Role'][i]} rdf:type famo:Record."+'\n')
    ttl_list.append(f"ex:wms_rolerec_{fakerole_df['Role'][i]} famo:hasInterpretation ex:wms_roleinterpretation_{fakerole_df['Role'][i]}."+'\n')
    #denotes corresponding dhid (a role, with an entity number)
    ttl_list.append(f"ex:wms_roleinterpretation_{fakerole_df['Role'][i]} famo:denotes ex:dhid_{fakerole_df['Role'][i]}."+'\n')
    ttl_list.append(f"ex:dhid_{fakerole_df['Role'][i]} rdf:type famo:Role."+'\n')
    ttl_list.append(f"ex:dhid_{fakerole_df['Role'][i]} famo:hasEntityNumber '{fakerole_df['EntityNum'][i]}'."+'\n')

    #record properties
    #subrole, if applicable
    if (pd.notna(fakerole_df['Subrole'][i])):
        ttl_list.append(f"ex:wms_rolerec_{fakerole_df['Role'][i]} famo:representsObjectOf ex:wms_roleincl_{fakerole_df['Role'][i]}_{dt_now}."+'\n')
        ttl_list.append(f"ex:wms_roleincl_{fakerole_df['Role'][i]}_{dt_now} a famo:RoleInclusionRelation."+'\n')
        ttl_list.append(f"ex:wms_roleincl_{fakerole_df['Role'][i]}_{dt_now} famo:subjectIsRepresentedBy ex:wms_rolerec_{fakerole_df['Subrole'][i]}."+'\n')

    #entity number
    ttl_list.append(f"ex:wms_rolerec_{fakerole_df['Role'][i]} famo:representsSubectOf ex:wms_eum_{fakerole_df['Role'][i]}_{dt_now}."+'\n')
    ttl_list.append(f"ex:wms_eum_{fakerole_df['Role'][i]}_{dt_now} a famo:EntityNumberProperty."+'\n')
    ttl_list.append(f"ex:wms_eum_{fakerole_df['Role'][i]}_{dt_now} famo:hasValue '{fakerole_df['EntityNum'][i]}'."+'\n')
    
    #description
    ttl_list.append(f"ex:wms_rolerec_{fakerole_df['Role'][i]} famo:representsSubectOf ex:wms_desc_{fakerole_df['Role'][i]}_{dt_now}."+'\n')
    ttl_list.append(f"ex:wms_desc_{fakerole_df['Role'][i]}_{dt_now} a famo:NameProperty."+'\n')
    ttl_list.append(f"ex:wms_desc_{fakerole_df['Role'][i]}_{dt_now} famo:hasValue '{fakerole_df['Description'][i]}'."+'\n')
    
    #status
    ttl_list.append(f"ex:wms_rolerec_{fakerole_df['Role'][i]} famo:representsSubectOf ex:wms_status_{fakerole_df['Role'][i]}_{dt_now}."+'\n')
    ttl_list.append(f"ex:wms_status_{fakerole_df['Role'][i]}_{dt_now} a famo:RoleStatusProperty."+'\n')
    ttl_list.append(f"ex:wms_status_{fakerole_df['Role'][i]}_{dt_now} famo:hasValue '{fakerole_df['RoleStatus'][i]}'."+'\n')  
    
    #criticality
    ttl_list.append(f"ex:wms_rolerec_{fakerole_df['Role'][i]} famo:representsSubectOf ex:wms_criticality_{fakerole_df['Role'][i]}_{dt_now}."+'\n')
    ttl_list.append(f"ex:wms_criticality_{fakerole_df['Role'][i]}_{dt_now} a famo:RoleCriticalityProperty."+'\n')
    ttl_list.append(f"ex:wms_criticality_{fakerole_df['Role'][i]}_{dt_now} famo:hasValue '{fakerole_df['Criticality'][i]}'."+'\n')  
    
    #space, if applicable
    if (pd.notna(fakerole_df['Space'][i])):
        ttl_list.append(f"ex:wms_rolerec_{fakerole_df['Role'][i]} famo:representsSubectOf ex:wms_space_{fakerole_df['Role'][i]}_{dt_now}."+'\n')
        ttl_list.append(f"ex:wms_space_{fakerole_df['Role'][i]}_{dt_now} a famo:RoleLocationRelation."+'\n')
        ttl_list.append(f"ex:wms_space_{fakerole_df['Role'][i]}_{dt_now} famo:objectIsRepresentedBy ex:wms_spacerec_{fakerole_df['Space'][i]}."+'\n')  
 

In [5]:
ttl_results = open("01_fakerole_out.ttl", "a")
#for i in range(0,len(fakeasset_df)-1):
#record instantiation and interpretation
ttl_results.writelines(ttl_list)

ttl_results.close()